In [1]:
from pathlib import Path
from typing import Dict, List, Sequence

from pydantic import BaseModel
from strictyaml import YAML, load

import regression_model

In [23]:
# Project Directories
PACKAGE_ROOT = Path(regression_model.__file__).resolve().parent
ROOT = PACKAGE_ROOT.parent
CONFIG_FILE_PATH = PACKAGE_ROOT / "config.yml"
DATASET_DIR = PACKAGE_ROOT / "datasets"
TRAINED_MODEL_DIR = PACKAGE_ROOT / "trained_models"

In [34]:
Path(regression_model.__file__).resolve()

WindowsPath('C:/Users/Aiden/Documents/Data_Science_Stuff/sf_Data_Science_Stuff/Practice/Udemy_1/Code/deploying-ml-models/section-05-production-model-package/regression_model/__init__.py')

In [32]:
Path(regression_model.__file__).resolve().parent

WindowsPath('C:/Users/Aiden/Documents/Data_Science_Stuff/sf_Data_Science_Stuff/Practice/Udemy_1/Code/deploying-ml-models/section-05-production-model-package/regression_model')

In [25]:
Path("AIDEN_PLAYGROUND").resolve()

WindowsPath('C:/Users/Aiden/Documents/Data_Science_Stuff/sf_Data_Science_Stuff/Practice/Udemy_1/Code/deploying-ml-models/section-05-production-model-package/AIDEN_PLAYGROUND')

In [26]:
Path("AIDEN_PLAYGROUND").resolve().parent

WindowsPath('C:/Users/Aiden/Documents/Data_Science_Stuff/sf_Data_Science_Stuff/Practice/Udemy_1/Code/deploying-ml-models/section-05-production-model-package')

In [27]:
ROOT

WindowsPath('C:/Users/Aiden/Documents/Data_Science_Stuff/sf_Data_Science_Stuff/Practice/Udemy_1/Code/deploying-ml-models/section-05-production-model-package')

In [28]:
CONFIG_FILE_PATH

WindowsPath('C:/Users/Aiden/Documents/Data_Science_Stuff/sf_Data_Science_Stuff/Practice/Udemy_1/Code/deploying-ml-models/section-05-production-model-package/regression_model/config.yml')

In [29]:
DATASET_DIR

WindowsPath('C:/Users/Aiden/Documents/Data_Science_Stuff/sf_Data_Science_Stuff/Practice/Udemy_1/Code/deploying-ml-models/section-05-production-model-package/regression_model/datasets')

In [30]:
TRAINED_MODEL_DIR

WindowsPath('C:/Users/Aiden/Documents/Data_Science_Stuff/sf_Data_Science_Stuff/Practice/Udemy_1/Code/deploying-ml-models/section-05-production-model-package/regression_model/trained_models')

In [35]:
class AppConfig(BaseModel):
    """
    Application-level config.
    """

    package_name: str
    training_data_file: str
    test_data_file: str
    pipeline_save_file: str

In [42]:
class ModelConfig(BaseModel):
    """
    All configuration relevant to model
    training and feature engineering.
    """

    target: str
    variables_to_rename: Dict
    features: List[str]
    test_size: float
    random_state: int
    alpha: float
    categorical_vars_with_na_frequent: List[str]
    categorical_vars_with_na_missing: List[str]
    numerical_vars_with_na: List[str]
    temporal_vars: List[str]
    ref_var: str
    numericals_log_vars: Sequence[str]
    binarize_vars: Sequence[str]
    qual_vars: List[str]
    exposure_vars: List[str]
    finish_vars: List[str]
    garage_vars: List[str]
    categorical_vars: Sequence[str]
    qual_mappings: Dict[str, int]
    exposure_mappings: Dict[str, int]
    garage_mappings: Dict[str, int]
    finish_mappings: Dict[str, int]

In [43]:
class Config(BaseModel):
    """Master config object."""

    app_config: AppConfig
    model_config: ModelConfig

In [44]:
def find_config_file() -> Path:
    """Locate the configuration file."""
    if CONFIG_FILE_PATH.is_file():
        return CONFIG_FILE_PATH
    raise Exception(f"Config not found at {CONFIG_FILE_PATH!r}")

In [45]:
def fetch_config_from_yaml(cfg_path: Path = None) -> YAML:
    """Parse YAML containing the package configuration."""

    if not cfg_path:
        cfg_path = find_config_file()

    if cfg_path:
        with open(cfg_path, "r") as conf_file:
            parsed_config = load(conf_file.read())
            return parsed_config
    raise OSError(f"Did not find config file at path: {cfg_path}")

In [46]:
def create_and_validate_config(parsed_config: YAML = None) -> Config:
    """Run validation on config values."""
    if parsed_config is None:
        parsed_config = fetch_config_from_yaml()

    # specify the data attribute from the strictyaml YAML type.
    _config = Config(
        app_config=AppConfig(**parsed_config.data),
        model_config=ModelConfig(**parsed_config.data),
    )

    return _config

In [ ]:
config = create_and_validate_config()

In [54]:
parsed_config = fetch_config_from_yaml()

In [57]:
parsed_config

YAML({'package_name': 'regression_model', 'training_data_file': 'train.csv', 'test_data_file': 'test.csv', 'target': 'SalePrice', 'pipeline_name': 'regression_model', 'pipeline_save_file': 'regression_model_output_v', 'variables_to_rename': {'1stFlrSF': 'FirstFlrSF', '2ndFlrSF': 'SecondFlrSF', '3SsnPorch': 'ThreeSsnPortch'}, 'features': ['MSSubClass', 'MSZoning', 'LotFrontage', 'LotShape', 'LandContour', 'LotConfig', 'Neighborhood', 'OverallQual', 'OverallCond', 'YearRemodAdd', 'RoofStyle', 'Exterior1st', 'ExterQual', 'Foundation', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'HeatingQC', 'CentralAir', 'FirstFlrSF', 'SecondFlrSF', 'GrLivArea', 'BsmtFullBath', 'HalfBath', 'KitchenQual', 'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageFinish', 'GarageCars', 'GarageArea', 'PavedDrive', 'WoodDeckSF', 'ScreenPorch', 'SaleCondition', 'YrSold'], 'test_size': '0.1', 'random_state': '0', 'alpha': '0.001', 'categorical_vars_with_na_frequent': ['BsmtQual', 'BsmtExposure', 'BsmtFi

In [73]:
app_config = AppConfig(**parsed_config.data)

In [76]:
app_config

AppConfig(package_name='regression_model', training_data_file='train.csv', test_data_file='test.csv', pipeline_save_file='regression_model_output_v')

<bound method BaseModel.schema of <class 'pydantic.main.BaseModel'>>